<b>Note: This Jupyter Notebook is associated with the article [The Options Wheel Strategy Explained (and How to Implement Using Python and Alpaca's Trading API)](https://alpaca.markets/learn/options-wheel-strategy).</b>

# Step 1: Setting Up the Environment and Trade Parameters
- Please use ``paper account``. Please ``DO NOT`` use this notebook with live account. In this notebook, we place orders for options as an example.

In [ ]:
# Install or upgrade the package `alpaca-py` and import it
!python3 -m pip install --upgrade alpaca-py

import pandas as pd
import numpy as np
from scipy.stats import norm
import alpaca
import time
from scipy.optimize import brentq
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit


from alpaca.trading.client import TradingClient
from alpaca.data.historical.option import OptionHistoricalDataClient
from alpaca.data.historical.stock import StockHistoricalDataClient, StockLatestTradeRequest
from alpaca.data.requests import StockBarsRequest, OptionLatestQuoteRequest, OptionChainRequest
from alpaca.trading.requests import GetOptionContractsRequest, MarketOrderRequest
from alpaca.trading.enums import AssetStatus, ContractType

In [ ]:
# API_KEY = "Alpaca's Trading API Key (Paper Account)"
# API_SECRET = "Alpaca's Trading API Secret Key (Paper Account)"

# A safe approach to setting up API credentials for Alpaca (Assume you run this notebook in Google Colab)
# Add your key to Colab Secrets. Add your API key to the Colab Secrets manager to securely store it
from google.colab import userdata
API_KEY = userdata.get('ALPACA_API_KEY')
API_SECRET = userdata.get('ALPACA_SECRET_KEY')
BASE_URL = None
## We use paper environment for this example
PAPER = True # Please do not modify this. This example is for paper trading only.

# Initialize Alpaca clients
trade_client = TradingClient(api_key=API_KEY, secret_key=API_SECRET, paper=PAPER, url_override=BASE_URL)
option_historical_data_client = OptionHistoricalDataClient(api_key=API_KEY, secret_key=API_SECRET, url_override=BASE_URL)
stock_data_client = StockHistoricalDataClient(api_key=API_KEY, secret_key=API_SECRET)

# Below are the variables for development this documents
# Please do not change these variables
trade_api_url = None
trade_api_wss = None
data_api_url = None
option_stream_data_wss = None

In [ ]:
# Configuration
underlying_symbol = 'WMT'

# Set the timezone
timezone = ZoneInfo("America/New_York")

# Get current date in US/Eastern timezone
today = datetime.now(timezone).date()

# Define a 5% range around the underlying price
STRIKE_RANGE = 0.05

# Buying power percentage to use for the trade
BUY_POWER_LIMIT = 0.10

# Risk free rate for the options greeks and IV calculations
RISK_FREE_RATE = 0.01

# Check account buying power
buying_power = float(trade_client.get_account().buying_power)

# Calculate the limit amount of buying power to use for the trade
buying_power_limit = buying_power * BUY_POWER_LIMIT

# Set the open interest volume threshold
OI_THRESHOLD = 200

# Set the expiration date range for the options
min_expiration = today + timedelta(days=7)
max_expiration = today + timedelta(days=35)

# Get the latest price of the underlying stock
def get_underlying_price(symbol):
    # Get current date in US/Eastern timezone
    today = datetime.now(timezone).date()
    # Get the latest trade for the underlying stock
    underlying_trade_request = StockLatestTradeRequest(symbol_or_symbols=symbol)
    underlying_trade_response = stock_data_client.get_stock_latest_trade(underlying_trade_request)
    return underlying_trade_response[symbol].price

# Get the latest price of the underlying stock
underlying_price = get_underlying_price(underlying_symbol)
print(f"{underlying_symbol} price: {underlying_price}")

# Set the minimum and maximum strike prices based on the underlying price
min_strike = str(underlying_price * (1 - STRIKE_RANGE))
max_strike = str(underlying_price * (1 + STRIKE_RANGE))

# Display the values
print(f"Underlying Symbol: {underlying_symbol}")
print(f"Strike Range: {STRIKE_RANGE}")
print(f"Buying Power Limit Percentage: {BUY_POWER_LIMIT}")
print(f"Risk Free Rate: {RISK_FREE_RATE}")
print(f"Account Buying Power: {buying_power}")
print(f"Buying Power Limit: {buying_power_limit}")
print(f"Open Interest Threshold: {OI_THRESHOLD}")
print(f"Minimum Expiration Date: {min_expiration}")
print(f"Maximum Expiration Date: {max_expiration}")
print(f"Minimum Strike Price: {min_strike}")
print(f"Maximum Strike Price: {max_strike}")

## Calculate Implied Volatility and Option Greek (delta)

In [ ]:
# Calculate implied volatility
def calculate_implied_volatility(option_price, S, K, T, r, option_type):

    # Define a reasonable range for sigma
    sigma_lower = 1e-6
    sigma_upper = 5.0  # Adjust upper limit if necessary

    # Check if the option is out-of-the-money and price is close to zero
    intrinsic_value = max(0, (S - K) if option_type == 'call' else (K - S))
    if option_price <= intrinsic_value + 1e-6:

        # print("Option price is close to intrinsic value; implied volatility is near zero.") # Uncomment for checking the status
        return 0.0

    # Define the function to find the root
    def option_price_diff(sigma):
        d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        if option_type == 'call':
            price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
        elif option_type == 'put':
            price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
        return price - option_price

    try:
        return brentq(option_price_diff, sigma_lower, sigma_upper)
    except ValueError as e:
        print(f"Failed to find implied volatility: {e}")
        return None

# Calculate option Delta
def calculate_delta(option_price, strike_price, expiry, underlying_price, risk_free_rate, option_type):
    T = (expiry - pd.Timestamp.now()).days / 365
    T = max(T, 1e-6)  # Set minimum T to avoid zero

    if T == 1e-6:
        print("Option has expired or is expiring now; setting delta based on intrinsic value.")
        if option_type == 'put':
            return -1.0 if underlying_price < strike_price else 0.0
        else:
            return 1.0 if underlying_price > strike_price else 0.0

    implied_volatility = calculate_implied_volatility(option_price, underlying_price, strike_price, T, risk_free_rate, option_type)
    print(f"implied volatility is {implied_volatility}")
    if implied_volatility is None or implied_volatility == 0.0:
        print("Implied volatility could not be determined, skipping delta calculation.")
        return None

    d1 = (np.log(underlying_price / strike_price) + (risk_free_rate + 0.5 * implied_volatility ** 2) * T) / (implied_volatility * np.sqrt(T))
    delta = norm.cdf(d1) if option_type == 'call' else -norm.cdf(-d1)
    return delta

# Step 1: Analyze the Underlying Asset

## Calculate ATR for the underlying stock

In [ ]:
# Get the historical data for the underlying stock by symbol
# ref. https://docs.alpaca.markets/reference/stockbars-1
def get_stock_data(underlying_symbol, stock_data_client, days=90):
    now = datetime.now(ZoneInfo("America/New_York"))
    req = StockBarsRequest(
        symbol_or_symbols=[underlying_symbol],
        timeframe=TimeFrame(amount=1, unit=TimeFrameUnit.Day),     # specify timeframe
        start=now - timedelta(days=days),                          # specify start datetime, default=the beginning of the current day.
    )
    return stock_data_client.get_stock_bars(req).df

# Calculate True Range for the underlying stock in the last 90 days
def calculate_true_range(data):
    data['prev_close'] = data['close'].shift(1)  # Shift the closing price by 1 day
    data['tr'] = np.maximum(
        data['high'] - data['low'],
        np.maximum(
            abs(data['high'] - data['prev_close']),
            abs(data['low'] - data['prev_close'])
        )
    )
    return data

# Calculate ATR for the underlying stock on the last 90 days
def calculate_atr(data, period=14):
    data['atr'] = data['tr'].rolling(window=period).mean()
    return data


def get_atr_for_stock(underlying_symbol, period=14):
    data = get_stock_data(underlying_symbol, stock_data_client, days=90)
    data = calculate_true_range(data)
    data = calculate_atr(data, period=period)
    return data[['tr', 'atr']].dropna()  # Return only TR and ATR columns

# Calculate 14-day Average True Ranges (ATRs) over 90 days
results = get_atr_for_stock(underlying_symbol=underlying_symbol, period=14)
atr_mean_90 = results["atr"].mean()
print(f"14-day ATR over 90 days is: {atr_mean_90}")

# Step 2: Sell Cash-Secured Puts (CSPs)

In [ ]:
# Check for put options
def get_put_options(underlying_symbol, min_strike, max_strike, min_expiration, max_expiration):

    # Fetch the options data to add to the portfolio
    req = GetOptionContractsRequest(underlying_symbols=[underlying_symbol],
                                    strike_price_gte=min_strike,
                                    strike_price_lte=max_strike,
                                    status=AssetStatus.ACTIVE,
                                    expiration_date_gte=min_expiration,
                                    expiration_date_lte=max_expiration,
                                    root_symbol=underlying_symbol,
                                    type=ContractType.PUT,
                                    )

    # Get put option chain of the underlying symbol
    put_options = trade_client.get_option_contracts(req).option_contracts
    return put_options

In [ ]:
get_put_options(underlying_symbol, min_strike, max_strike, min_expiration, max_expiration)

In [ ]:
# Find the appropriate short put option to sell based on the delta and buying power limit
def find_short_put_option(put_options, underlying_price, risk_free_rate, buying_power_limit):

    short_put = None

    for option_data in put_options:
        try:
            # Ensure open_interest is valid and open_interest_date exists
            if option_data.open_interest is None or option_data.open_interest_date is None:
                # Skip options with missing open interest or quote data
                continue

            # Check open interest meets the threshold
            if float(option_data.open_interest) <= OI_THRESHOLD:
                print(f"Skipping option {option_data.symbol} due to lack of open interest. You may change the OI_THRESHOLD") # Uncomment for checking the status
                continue

            # Get the latest quote for the option price
            option_symbol = option_data.symbol
            option_quote_request = OptionLatestQuoteRequest(symbol_or_symbols=option_symbol)
            option_quote = option_historical_data_client.get_option_latest_quote(option_quote_request)[option_symbol]

            # Extract option details
            option_price = (option_quote.bid_price + option_quote.ask_price) / 2
            option_size = float(option_data.size)
            strike_price = float(option_data.strike_price)
            expiry = pd.Timestamp(option_data.expiration_date)

            print(f"option_symbol is {option_symbol}")
            print(f"option_price is {option_price}")
            print(f"strike price is {strike_price}")

            # Calculate delta for each option
            delta = calculate_delta(
                option_price=option_price,
                strike_price=strike_price,
                expiry=expiry,
                underlying_price=underlying_price,
                risk_free_rate=risk_free_rate,
                option_type='put'
            )
            print(f"delta is {delta}")

            # Check if delta is between -0.42 and -0.18 and if the total contract exceeds the buying power limit
            if delta is not None and delta >= -0.42 and delta <= -0.18 and strike_price * option_size < buying_power_limit:

                # Create the appropriate short put option in the dictionary format
                short_put = {
                    'close_price': option_data.close_price,
                    'close_price_date': option_data.close_price_date,
                    'expiration_date': option_data.expiration_date,
                    'id': option_data.id,
                    'name': option_data.name,
                    'open_interest': option_data.open_interest,
                    'open_interest_date': option_data.open_interest_date,
                    'root_symbol': option_data.root_symbol,
                    'size': option_data.size,
                    'status': option_data.status,
                    'strike_price': option_data.strike_price,
                    'style': option_data.style,
                    'symbol': option_data.symbol,
                    'tradable': option_data.tradable,
                    'type': option_data.type,
                    'underlying_asset_id': option_data.underlying_asset_id,
                    'underlying_symbol': option_data.underlying_symbol,
                    'initial_delta': delta,
                    'initial_option_price': option_price,
                }

                print(f"short_put is {short_put}")
            # If the short put is found, break the loop
            if short_put:
                break

        except KeyError as e:
            print(f"Missing data for option {option_symbol}: {e}")
            continue

    return short_put

In [ ]:
def execute_cash_secured_put(underlying_symbol, risk_free_rate, buying_power_limit, min_strike, max_strike, min_expiration, max_expiration):
    # Get put options
    put_options = get_put_options(underlying_symbol, min_strike, max_strike, min_expiration, max_expiration)

    if put_options:
        # Get the latest price of the underlying stock
        underlying_price = get_underlying_price(symbol=underlying_symbol)

        # Find appropriate short put options
        short_put = find_short_put_option(put_options, underlying_price, risk_free_rate, buying_power_limit)

        # Proceed if short put options are found
        if short_put:

            # Place orders for the strategy
            # Sell the -20 ~ -40 delta put
            req = MarketOrderRequest(
                symbol=short_put['symbol'],
                qty=1,
                side='sell',
                type='market',
                time_in_force='day'
            )

            trade_client.submit_order(req)

            success_message = (f"Placing Cash Secured Put on {underlying_symbol} successfully:\n"
                               f"Sell {short_put['initial_delta']} Delta Put: {short_put['symbol']} "
                               f"(Strike: {short_put['strike_price']}, Premium to Receive: {short_put['initial_option_price']})")
            return success_message, short_put

        else:
            return "Could not find suitable options which has delta between -20 and -40.", None
    else:
        return "No put options available for the underlying symbol.", None

In [ ]:
# You can run the `find_short_put_option` function just to find the short put option.
put_options = get_put_options(underlying_symbol, min_strike, max_strike, min_expiration, max_expiration)
short_put_list = find_short_put_option(put_options, underlying_price, RISK_FREE_RATE, buying_power_limit)
short_put_list

In [ ]:
# Run the `execute_cash_secured_put` function to execute the cash secured put
message, short_put = execute_cash_secured_put(underlying_symbol, RISK_FREE_RATE, buying_power_limit, min_strike, max_strike, min_expiration, max_expiration)
message, short_put

# Step 3: Sell Covered Calls (CCs)

The upper Bollinger Band for the covered call strike price

In [ ]:
# setup stock historical data client
now = datetime.now(ZoneInfo("America/New_York"))
req = StockBarsRequest(
    symbol_or_symbols=[underlying_symbol],
    timeframe=TimeFrame(amount=1, unit=TimeFrameUnit.Day),  # specify timeframe
    start=today-timedelta(days=60),                       # specify start datetime, default=the beginning of the current day.
    end=today,                                                  # specify end datetime, default=now
)

stock_data = stock_data_client.get_stock_bars(req).df


# Define the window period for the Bollinger Bands
window = 20

# Calculate the Simple Moving Average (SMA)
stock_data['SMA'] = stock_data['close'].rolling(window=window).mean()

# Calculate the rolling standard deviation
stock_data['StdDev'] = stock_data['close'].rolling(window=window).std()

# Set the multiplier (commonly 2)
multiplier = 2

# Calculate the Upper Bollinger Band
stock_data['Upper_Band'] = stock_data['SMA'] + (multiplier * stock_data['StdDev'])

# Get the most recent Upper Band value
latest_upper_bollinger_band = stock_data['Upper_Band'].iloc[-1]

print(f"Latest Upper Bollinger Band is: {latest_upper_bollinger_band}")

Check sufficient shares for the covered call

In [ ]:
# Check if the portfolio has sufficient shares of the underlying stock/ETF to execute a covered call
def has_sufficient_shares(symbol, required_qty=100):

    # Fetch all positions
    positions = trade_client.get_all_positions()

    # Iterate through positions to check the stock symbol
    for position in positions:
        if position.symbol == symbol and position.asset_class == 'us_equity':
            # Check if the available quantity is greater than or equal to the required quantity
            if int(position.qty_available) >= required_qty:
                print(f"Sufficient shares available for {symbol}.")
                return True
            else:
                print(f"Insufficient shares for {symbol}. Required: {required_qty}, Available: {position.qty_available}")
                return False

    print(f"No position found for the stock: {symbol}")
    return False

# If the portfolio has sufficient shares, run the following codes to execute the covered call
has_sufficient_shares(underlying_symbol)

Check the call option chains with a strike price above the purchased put option's strike price and below the predefined maximum strike price.

In [ ]:
# Check for call options which are above the short_put strike price
def get_call_options(underlying_symbol, option_data, max_strike, min_expiration, max_expiration):

    # Check if the purchased option strike (=option_data['strike_price']) in the CSP is less than max_strike
    if option_data['strike_price'] >= float(max_strike):
        return f"Option with strike price {option_data.strike_price} exceeds the max strike price {max_strike}."

    # Fetch the options data to add to the portfolio
    req = GetOptionContractsRequest(underlying_symbols=[underlying_symbol],
                                    strike_price_gte=str(option_data['strike_price']),
                                    strike_price_lte=max_strike,
                                    status=AssetStatus.ACTIVE,
                                    expiration_date_gte=min_expiration,
                                    expiration_date_lte=max_expiration,
                                    root_symbol=underlying_symbol,
                                    type=ContractType.CALL,
                                    )

    # Get put option chain of the underlying symbol
    call_options = trade_client.get_option_contracts(req).option_contracts
    return call_options

Narrow down the `call_options` to the one that meet the predefined criteria and output the final call option chain.

In [ ]:
# Find the appropriate short call option to sell based on the delta and buying power limit
def find_short_call_option(call_options, underlying_price, risk_free_rate, latest_upper_boiler_band):

    short_call = None

    for option_data in call_options:
        try:
            # Ensure open_interest is valid and open_interest_date exists
            if option_data.open_interest is None or option_data.open_interest_date is None:
                # Skip options with missing open interest or quote data
                continue

            # Check open interest meets the threshold
            if float(option_data.open_interest) <= OI_THRESHOLD:
                # print(f"Skipping option {option_name} due to missing trade or quote data.") # Uncomment for checking the status
                continue

            # Get the latest quote for the option price
            option_symbol = option_data.symbol
            option_quote_request = OptionLatestQuoteRequest(symbol_or_symbols=option_symbol)
            option_quote = option_historical_data_client.get_option_latest_quote(option_quote_request)[option_symbol]

            # Extract option details
            option_price = (option_quote.bid_price + option_quote.ask_price) / 2
            option_size = float(option_data.size)
            strike_price = float(option_data.strike_price)
            expiry = pd.Timestamp(option_data.expiration_date)

            print(f"option_symbol is {option_symbol}")
            print(f"option_price is {option_price}")
            print(f"strike price is {strike_price}")

            # Calculate delta for each option
            delta = calculate_delta(
                option_price=option_price,
                strike_price=strike_price,
                expiry=expiry,
                underlying_price=underlying_price,
                risk_free_rate=risk_free_rate,
                option_type='call'
            )
            print(f"delta is {delta}")

            # Check if delta is between 0.42 and 0.18 and if the strike price is greater than the latest upper boiler band
            if delta is not None and delta >= 0.18 and delta <= 0.42 and strike_price > latest_upper_boiler_band:

                # Create the appropriate short put option in the dictionary format
                short_call = {
                    'close_price': option_data.close_price,
                    'close_price_date': option_data.close_price_date,
                    'expiration_date': option_data.expiration_date,
                    'id': option_data.id,
                    'name': option_data.name,
                    'open_interest': option_data.open_interest,
                    'open_interest_date': option_data.open_interest_date,
                    'root_symbol': option_data.root_symbol,
                    'size': option_data.size,
                    'status': option_data.status,
                    'strike_price': option_data.strike_price,
                    'style': option_data.style,
                    'symbol': option_data.symbol,
                    'tradable': option_data.tradable,
                    'type': option_data.type,
                    'underlying_asset_id': option_data.underlying_asset_id,
                    'underlying_symbol': option_data.underlying_symbol,
                    'initial_delta': delta,
                    'initial_option_price': option_price,
                }

                print(f"short_call is {short_put}")
            # If the short put is found, break the loop
            if short_call:
                break

        except KeyError as e:
            print(f"Missing data for option {option_symbol}: {e}")
            continue

    return short_call

Execute the covered call strategy based on the selected call option chain by the `find_short_call_option` function.

In [ ]:
def execute_covered_call(underlying_symbol, risk_free_rate, latest_upper_boiler_band, option_data, max_strike, min_expiration, max_expiration):

    """
    Execute a covered call strategy if sufficient shares are available.
    """

    required_qty = 100

    if not has_sufficient_shares(underlying_symbol, required_qty):

        return f"Insufficient shares for {underlying_symbol} to execute a covered call.", None

    # Get call options
    call_options = get_call_options(underlying_symbol, option_data, max_strike, min_expiration, max_expiration)

    if call_options:
        # Get the latest price of the underlying stock
        underlying_price = get_underlying_price(symbol=underlying_symbol)

        # Find appropriate short put options
        short_call = find_short_call_option(call_options, underlying_price, risk_free_rate, latest_upper_boiler_band)

        # Proceed if short call options are found
        if short_call:

            # Place orders for the covered call if delta is between 0.42 and 0.18 and if the strike price is greater than the latest upper boiler band
            # Sell the 20 ~ 40 delta call
            req = MarketOrderRequest(
                symbol=short_call['symbol'],
                qty=1,
                side='sell',
                type='market',
                time_in_force='day'
            )
            trade_client.submit_order(req)

            success_message = (f"Placing Coverd Call on {underlying_symbol} successfully:\n"
                               f"Sell {short_call['delta']} Delta Put: {short_call['symbol']} "
                               f"(Strike: {short_call['strike_price']}, Premium to Receive: {short_call['option_price']})")
            return success_message, short_call

        else:
            return "Could not find suitable options which has delta between 20 and 40.", None
    else:
        return "No call options available for the underlying symbol.", None

In [ ]:
# Run the `execute_covered_call` function to execute the covered call
message, short_call = execute_covered_call(underlying_symbol, RISK_FREE_RATE, latest_upper_bollinger_band, short_put, max_strike, min_expiration, max_expiration)
message, short_call

# Step 4: Roll or Rinse the Options (Optional)

roll_rinse_execution can be used for both cash secured puts and covered calls. By setting rolling=True, the function roll the strategy by buying either the short put or the short call and selling another option of the same underlying with a different strike and/or expiration date.

In [ ]:
# Exit the market order
def roll_rinse_execution(option_data, rolling=True):

    # if rolling the option, close the short put and re-enter the market with a new cash secured put or close the long call and re-enter the market with a new long call
    if rolling:
        # Deternine if the option is a call or put
        option_type = option_data['type'].value

        # If the option is a put, close the short put by buying it back
        if option_type == 'put':

            # Close the short put by buying it back
            req = MarketOrderRequest(
                symbol=option_data['symbol'],
                qty=1,
                side='buy',
                type='market',
                time_in_force='day'
            )

            # Submit the order to close the short put
            trade_client.submit_order(req)
            print(f"Closed short {option_type} option: {option_data['symbol']} bought")

            # Re-enter the market with a new cash secured put
            rolling_message, short = execute_cash_secured_put(underlying_symbol, RISK_FREE_RATE, buying_power_limit)

            if short:
                # You can add the `rolling_message` from the `execute_cash_secured_put` function below to check if the short put or call is not sccessfully placed
                return f"Re-entering market with new cash secured put on {option_data['underlying_symbol']}", short
            else:
                return f"Failed to re-enter market with new cash secured put on {option_data['underlying_symbol']}", None

        # If the option is a call, close the short call by buying it back
        else:
            # Close the short call by buying it back
            req = MarketOrderRequest(
                symbol=option_data['symbol'],
                qty=1,
                side='buy',
                type='market',
                time_in_force='day'
            )

            # Submit the order to close the covered call (short call)
            trade_client.submit_order(req)
            print(f"Closing short {option_type} option: {option_data['symbol']} sold")

            # Re-enter the market with a new covered call
            rolling_message, short = execute_covered_call(underlying_symbol, RISK_FREE_RATE, buying_power_limit)

            if short:
                return f"Re-entering market with new covered call on {option_data['underlying_symbol']}", short
            else:
                return f"Failed to re-enter market with new covered call on {option_data['underlying_symbol']}", None

    else:
        # If the option is a put, close the short put by buying it back
        if option_type == 'put':

            # Close the short put by buying it back
            req = MarketOrderRequest(
                symbol=option_data['symbol'],
                qty=1,
                side='buy',
                type='market',
                time_in_force='day'
            )
            trade_client.submit_order(req)
            return f"Closed short {option_type} option: {option_data['symbol']} bought", None

         # If the option is a call, close the short call by buying it back
        else:
            # Close the short call by buying it back
            req = MarketOrderRequest(
                symbol=option_data['symbol'],
                qty=1,
                side='buy',
                type='market',
                time_in_force='day'
            )

            # Submit the order to close the covered call (short call)
            trade_client.submit_order(req)
            return f"Closing short {option_type} option: {option_data['symbol']} sold", None


In [ ]:
# calculate the current delta of the option (rolling or rinsing)
def roll_rinse_option(option_data, rolling=True):

    # Get the latest quote for the option price
    option_symbol = option_data["symbol"]
    option_quote_request = OptionLatestQuoteRequest(symbol_or_symbols=option_symbol)
    option_quote = option_historical_data_client.get_option_latest_quote(option_quote_request)[option_symbol]

    # Extract option details
    current_option_price = (option_quote.bid_price + option_quote.ask_price) / 2
    strike_price = float(option_data["strike_price"])
    expiry = pd.Timestamp(option_data["expiration_date"])

    print(f"option_symbol is {option_symbol}")
    print(f"current option_price is {current_option_price}")
    print(f"current strike price is {strike_price}")

    # Deternine if the option is a call or put
    option_type = option_data['type'].value
    # If the option is a put, calculate the delta for the put option
    if option_type == 'put':

        # Calculate delta for each option
        current_delta = calculate_delta(
            option_price=current_option_price,
            strike_price=strike_price,
            expiry=expiry,
            underlying_price=underlying_price,
            risk_free_rate=RISK_FREE_RATE,
            option_type='put'
        )

    # If the option is a call, calculate the delta for the call option
    else:
        current_delta = calculate_delta(
            option_price=current_option_price,
            strike_price=strike_price,
            expiry=expiry,
            underlying_price=underlying_price,
            risk_free_rate=RISK_FREE_RATE,
            option_type='call'
        )

    # Set target profit levels in two ways: 1) 50% of the initial credit received, 2) 2x the initial delta of the short put
    target_profit_price = option_data['initial_option_price'] * 0.5  # 50% of credit received
    initial_delta = option_data['initial_delta'] * 2  # Set target delta level at 2x the initial delta of the short put

    # roll or rinse the option if the absoluete value of the current delta is greater than or equal to the initial delta
    if abs(current_delta) >= abs(initial_delta) or current_option_price <= target_profit_price:

        # Roll or rinse the option
        rinsing_message, short = roll_rinse_execution(option_data, rolling=rolling)

        # you can add the `rinsing_message` from the `roll_rinse_execution` function below to check if the short put or call is not sccessfully placed
        return f"Current delta {current_delta} exceeds twice the initial delta {initial_delta} or the option price is less than 50% of the initial credit received. Executing roll/rinse.", short

    else:
        return f"Current delta {current_delta} is less than twice the initial delta {initial_delta} and the option price is greater than 50% of the initial credit received. Holding the position.", None


## Roll or Rinse Positions

In [ ]:
message, short = roll_rinse_option(option_data=short_put, rolling=True)
message, short